In [ ]:
!pip install tensorflow

In [2]:
import tensorflow as tf

# Making multiple image tiles from single tile

In [ ]:
!pip install geotile==0.2.3

In [4]:
from geotile import GeoTile

Connect Google Drive

In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Creating tiles in several folders at once

In [6]:
import os

Five tiles are kept in five folders and five mask files are kept in five folders.

In [ ]:
folder_paths = [
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/One',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/Two',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Tiles/Three',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/One',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/Three',
    '/content/gdrive/MyDrive/Colab Notebooks/pc/Masks/Two'
    # Add more folder paths as needed
]
# Iterate over each folder path and generate the tiles
for folder_path in folder_paths:
    os.chdir(folder_path)
    image_files = os.listdir(folder_path)
    # Create the output folder corresponding to the input folder
    output_folder = os.path.join(folder_path, 'output')
    os.makedirs(output_folder, exist_ok=True)

    for image_file in image_files:
        if image_file.endswith('.tif'):  # Modify the extension if needed
            gt = GeoTile(image_file)
            gt.generate_tiles(output_folder)


Tiles from folder number one, two, four, and five are used for training and therefore are stored those in a sigle folder named 'train'. Tiles from the folder number three are used for testing and therefore are kept in the folder named 'test'. Both in train and test folders, there are two folders. One (images)is for storing the images and another (masks) is for storing the masks.

Checking the information of the image

In [ ]:
import os
os.chdir(r"/content/gdrive/MyDrive/Colab Notebooks/pc1/train")
gt = GeoTile('10_i.tif')

In [ ]:
gt.height

256

In [ ]:
gt.width

256

In [ ]:
gt.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 256,
 'height': 256,
 'count': 3,
 'crs': CRS.from_epsg(32611),
 'transform': Affine(0.5, 0.0, 464525.3746,
        0.0, -0.5, 5920603.0729)}

# Import necessary libraries

In [7]:
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

Connect google drive

In [8]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Set the working directory

In [9]:
import os
os.chdir(r'/content/gdrive/MyDrive/Colab Notebooks/pc1')                  #Respective folder(s) in the drive

# Data Preprocessing

Now, all the images and masks  for training are stored in one folder named 'training', and all the images and masks  for testing are stored in another folder named 'testing'. These training and testing folders are kept inside the 'Modeling' folder.

In [ ]:
train_x = sorted(glob.glob('train/*_i.tif'))
train_y = sorted(glob.glob('train/*_l.tif'))
test_x = sorted(glob.glob('test/*_i.tif'))
test_y = sorted(glob.glob('test/*_l.tif'))

print(len(train_x), len(test_x))

100 25


Converting training image to numpy array

In [ ]:
train_xx = np.zeros((100, 256, 256, 3))
train_yy = np.zeros((100, 256, 256, 1))
for i, (img, mask) in enumerate(zip(train_x, train_y)):
  # if i == 1:
  #   break

  img = Image.open(img)
  np_img = np.array(img)
  train_xx[i] = np_img

  mask = Image.open(mask)
  np_mask = np.array(mask).reshape(256, 256, 1)
  train_yy[i] = np_mask

Converting testing image to numpy array

In [ ]:
test_xx = np.zeros((25, 256, 256, 3))
test_yy = np.zeros((25, 256, 256, 1))
for i, (img, mask) in enumerate(zip(test_x, test_y)):

  img = Image.open(img)
  np_img = np.array(img)
  test_xx[i] = np_img

  mask = Image.open(mask)
  np_mask = np.array(mask).reshape(256, 256, 1)
  test_yy[i] = np_mask

Let's plot a sample input RGB image and output image with classes

In [ ]:
plt.imshow(train_xx[20, :, :, :].astype('uint8'))
plt.show()
plt.imshow(train_yy[20, :, :, 0].astype('uint8'))
plt.show()

# Save to numpy format

In [ ]:
train_xx.shape

(50, 256, 256, 3)

In [ ]:
train_yy.shape

(50, 256, 256, 1)

In [ ]:
np.save('train_xx.npy', train_xx)
np.save('train_yy.npy', train_yy)
np.save("test_xx.npy", test_xx)
np.save("test_yy.npy", test_yy)

# Data for land use land cover mapping

Input data are RGB satellite images. And output are images of land cover type. There are 6 land cover types as below,

- Pixel value 1: Road
- Pixel value 2: Building
- Pixel value 3: Needle leaf
- Pixel value 4: Broad leaf
- Pixel value 5: Barren land
- Pixel value 6: Unlabelled


Set the working directory

In [10]:
import os
os.chdir(r'/content/gdrive/MyDrive/Colab Notebooks/pc1')                    #Respective folder(s) in the drive

Data is already randomized and split in to training / test sets. So we can go ahead and use them as it is.

In [11]:
x_train = np.load('train_xx.npy').astype('float32')
y_train= np.load('train_yy.npy').astype('float32')
x_test = np.load('test_xx.npy').astype('float32')
y_test = np.load('test_yy.npy').astype('float32')

print("x_train shape", x_train.shape)
print("y_train shape", y_train.shape)
print("y_test shape", x_test.shape)
print("y_test shape", y_test.shape)

x_train shape (100, 256, 256, 3)
y_train shape (100, 256, 256, 1)
y_test shape (25, 256, 256, 3)
y_test shape (25, 256, 256, 1)


Let's plot a sample input RGB image and output image with land cover

In [ ]:
plt.imshow(x_test[3,:,:,:].astype('uint8'))
plt.show()

plt.imshow(y_test[3,:,:,0].astype('uint8'))
plt.show()

Since land cover data include classes, let's perform one-hot encoding first.

In [12]:
from tensorflow.keras.utils import to_categorical

y_train_1hot = to_categorical(y_train)
y_test_1hot = to_categorical(y_test)

y_train_1hot.shape

(100, 256, 256, 7)

# Model development (Unet)

In [13]:
x_in = Input(shape=(256, 256, 3))

'''Encoder'''
x_temp = Conv2D(32, (3, 3), activation='relu', padding='same')(x_in)
x_temp = Dropout(0.25)(x_temp)
x_skip1 = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip1)
x_temp = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.25)(x_temp)
x_skip2 = Conv2D(32, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip2)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.25)(x_temp)
x_skip3 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = MaxPooling2D((2,2))(x_skip3)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2D(64, (3, 3), activation='relu', padding='same')(x_temp)

'''Decoder'''
x_temp = Conv2DTranspose(64, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu',  padding='same')(x_temp)
x_temp = Concatenate()([x_temp, x_skip3])
x_temp = Conv2DTranspose(64, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu',  padding='same')(x_temp)
x_temp = Concatenate()([x_temp, x_skip2])
x_temp = Conv2DTranspose(32, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu',  padding='same')(x_temp)
x_temp = Concatenate()([x_temp, x_skip1])
x_temp = Conv2DTranspose(32, (3, 3), activation='relu',  padding='same')(x_temp)
x_temp = Dropout(0.5)(x_temp)
x_temp = Conv2DTranspose(32, (3, 3), activation='relu',  padding='same')(x_temp)

'''Use 1 by 1 Convolution to get desired output bands'''
x_temp = Conv2D(32, (1, 1), activation='relu', padding='same')(x_temp)
x_temp = Conv2D(32, (1, 1), activation='relu', padding='same')(x_temp)
x_out = Conv2D(7, (1, 1), activation='sigmoid', padding='same')(x_temp)

model = Model(inputs=x_in, outputs=x_out)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 256, 256, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
history = model.fit(x_train, y_train_1hot, validation_data=(x_test, y_test_1hot), epochs=110, batch_size=18, verbose=1)

Plot the loss

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Accuracy over epoch')
ax1.legend(['Train', 'Test'], loc='upper right')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Loss over epoch')
ax2.legend(['Train', 'Test'], loc="upper right")

plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/pc1/figure.png')                   #Respective folder(s) in the drive

#Prediction

In [15]:
'''Prediction over the test dataset'''
pred_test = model.predict(x_test)

pred_test = np.argmax(pred_test, axis=-1)
print(pred_test.shape)

1/1 [==============================] - 4s 4s/step
(25, 256, 256)


In [ ]:
from PIL import Image
import numpy as np
import os

# Create a directory to store the predicted images
save_dir = '/content/gdrive/MyDrive/Colab Notebooks/predicted_images_withoutAugmentation'
os.makedirs(save_dir, exist_ok=True)

# Save each predicted image to the directory
for i in range(pred_test.shape[0]):
    image = pred_test[i, :, :]
    image = np.uint8(image * 255)  # Convert to uint8
    image = np.squeeze(image)  # Remove the single-channel dimension if present
    image_path = os.path.join(save_dir, f'predicted_image_{i}.png')
    Image.fromarray(image).save(image_path)

print("Predicted images saved successfully.")

Predicted images saved successfully.


let's compare sample predicted and actual land cover image with input RGB image

In [ ]:
plt.imshow(pred_test[23, :, :])
plt.show()
plt.imshow(y_test[23, :, :, 0])
plt.show()
plt.imshow(x_test[23,:,:,:].astype('uint8'))
plt.show()